Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.2 MB/s eta 0:00:00


QUANTUM FOURIER TRANSFORM (QFT)

QuTiP ile QFT (matematiksel tanım)

In [2]:
import numpy as np
from qutip import Qobj, qeye, tensor, basis, hadamard_transform

# n qubit sayısı
n = 3
N = 2**n

# QFT operatörü tanımı
def qft_matrix(n):
    N = 2**n
    omega = np.exp(2j * np.pi / N)
    F = np.array([[omega**(j*k) for k in range(N)] for j in range(N)]) / np.sqrt(N)
    return Qobj(F)

# 3 qubitlik QFT operatörü
QFT = qft_matrix(3)

# Örnek durum |3> = |011>
state = basis(N, 3)
transformed = QFT * state

print("Girdi durumu |3> = |011> :\n", state.full().T)
print("\nQFT(|3>):\n", np.round(transformed.full(), 3))


ImportError: cannot import name 'hadamard_transform' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

Qiskit ile QFT devresi

In [3]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
import numpy as np

# 3 qubit QFT devresi oluştur
n = 3
qc = QuantumCircuit(n)

# Giriş durumu |3> = |011>
qc.x(0)
qc.x(1)

# QFT uygula
qc.append(QFT(num_qubits=n, do_swaps=True), range(n))
qc.measure_all()

# Simülasyon
sim = Aer.get_backend('qasm_simulator')
result = sim.run(assemble(transpile(qc, sim))).result()
counts = result.get_counts()

print("QFT çıktı histogramı:")
print(counts)
qc.draw('mpl')


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

Cirq ile QFT devresi (elle inşa edilmiş)

In [4]:
import cirq
import numpy as np

# Yardımcı fonksiyonlar
def qft_circuit(qubits):
    circuit = cirq.Circuit()
    n = len(qubits)
    for i in range(n):
        circuit.append(cirq.H(qubits[i]))
        for j in range(i+1, n):
            angle = np.pi / (2 ** (j - i))
            circuit.append(cirq.CZ(qubits[j], qubits[i])**(1 / (2 ** (j - i))))
    circuit.append([cirq.SWAP(qubits[i], qubits[n - i - 1]) for i in range(n//2)])
    return circuit

# 3 qubit
qubits = [cirq.LineQubit(i) for i in range(3)]
circuit = qft_circuit(qubits)

print("QFT Devresi:")
print(circuit)


QFT Devresi:
                  ┌───────┐
0: ───H───@────────@──────────────────────×───
          │        │                      │
1: ───────@^0.5────┼─────H────@───────────┼───
                   │          │           │
2: ────────────────@^0.25─────@^0.5───H───×───
                  └───────┘


QUANTUM WAVELET TRANSFORM

Cirq ile Quantum Wavelet Transform

In [8]:
import cirq
import numpy as np

# --- 1. Qubit tanımla ---
qubits = [cirq.LineQubit(i) for i in range(2)]

# --- 2. QWT devresi (Haar benzeri) ---
circuit = cirq.Circuit()
circuit.append(cirq.X(qubits[0]))  # |01> durumu hazırla
circuit.append(cirq.H(qubits[1]))  # Haar transform başlat
circuit.append(cirq.CNOT(qubits[1], qubits[0]))  # kontrollü işlem

# --- 3. Ölçüm ---
circuit.append(cirq.measure(*qubits, key='result'))

# --- 4. Simülasyon ---
sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)

print("QWT devresi:")
print(circuit)
print("\nSonuçlar:")
print(result.histogram(key='result'))


QWT devresi:
0: ───X───X───M('result')───
          │   │
1: ───H───@───M─────────────

Sonuçlar:
Counter({2: 528, 1: 472})


Qiskit ile Quantum Wavelet Transform

In [7]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
import numpy as np

# --- 1. 2-qubitlik devre ---
qc = QuantumCircuit(2)

# --- 2. Giriş durumu |01> ---
qc.x(0)

# --- 3. Haar wavelet benzeri dönüşüm (Hadamard uygulaması) ---
qc.h(1)
qc.cx(1, 0)

# --- 4. Ölçüm ---
qc.measure_all()

# --- 5. Simülasyon ---
backend = Aer.get_backend("qasm_simulator")
result = backend.run(assemble(transpile(qc, backend))).result()
counts = result.get_counts()

print("QWT çıktı histogramı:")
print(counts)
qc.draw('mpl')


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Quantum Wavelet Transform (QWT)

In [6]:
import numpy as np
from qutip import Qobj, tensor, qeye, basis, hadamard_transform

# --- 1. Haar wavelet matrisi tanımı (2 boyut için) ---
haar_wavelet = (1/np.sqrt(2)) * np.array([[1, 1],
                                           [1, -1]])

# --- 2. 2-qubit için genişletilmiş wavelet operatörü ---
QWT = Qobj(tensor(haar_wavelet, haar_wavelet))

# --- 3. Giriş durumu (örnek sinyal) ---
# |f> = |01>
f = tensor(basis(2, 0), basis(2, 1))

# --- 4. QWT dönüşümü ---
transformed = QWT * f

print("Girdi durumu |f> = |01> :")
print(f.full().T)
print("\nQWT(|01>) sonucu:")
print(np.round(transformed.full(), 3))


ImportError: cannot import name 'hadamard_transform' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

QUANTUM SIGNAL PROCESSİNG ALGORİTMALARI

QSP – Cirq ile Örnek

In [9]:
import cirq
import numpy as np

# Tek qubit ve parametreli unitary (sinyal operatörü)
q = cirq.GridQubit(0, 0)
theta = np.pi / 4  # örnek sinyal parametresi

# W(θ) = e^{iθX}
W = cirq.rx(2 * theta)

# Faz açıları (örnek)
phi = [0.3, 1.1, 0.7]

# QSP devresi oluşturma
circuit = cirq.Circuit()
for i, p in enumerate(phi):
    circuit.append(cirq.rz(2 * p)(q))
    circuit.append(W(q))

# Devreyi çiz
print("=== QSP Circuit (Cirq) ===")
print(circuit)

# Simülasyon
sim = cirq.Simulator()
result = sim.simulate(circuit)
print("\nFinal state vector:")
print(np.round(result.final_state_vector, 3))


=== QSP Circuit (Cirq) ===
(0, 0): ───Rz(0.191π)───Rx(0.5π)───Rz(0.7π)───Rx(0.5π)───Rz(0.446π)───Rx(0.5π)───

Final state vector:
[-0.826-0.465j -0.216+0.235j]


QSP – Qiskit ile Örnek

In [10]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import RXGate, RZGate
import numpy as np

theta = np.pi / 4
phi = [0.3, 1.1, 0.7]

qc = QuantumCircuit(1)

for p in phi:
    qc.append(RZGate(2 * p), [0])
    qc.append(RXGate(2 * theta), [0])

qc.draw('mpl')

# Simülasyon
backend = Aer.get_backend('statevector_simulator')
result = execute(qc, backend).result()
statevector = result.get_statevector(qc)

print("=== QSP Circuit (Qiskit) ===")
print(qc)
print("\nFinal statevector:", np.round(statevector, 3))


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QSP – QuTiP ile Örnek

In [11]:
import numpy as np
from qutip import Qobj, rx, rz, tensor, basis

# Parametreler
theta = np.pi / 4
phi = [0.3, 1.1, 0.7]

# Başlangıç durumu |0>
psi0 = basis(2, 0)

# W(θ) = e^{iθX} (rx)
W = rx(2 * theta)

# QSP operatörünün oluşturulması
U = Qobj(np.eye(2))
for p in phi:
    U = rz(2 * p) * W * U

# U’nun uygulanması
psi_final = U * psi0

print("=== QSP Operator (QuTiP) ===")
print(U)
print("\nFinal state:")
print(psi_final)


ImportError: cannot import name 'rx' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)